In [1]:
import matplotlib.pyplot as plt
from shapely import geometry
from shapely import wkt
import csv
import numpy as np
from time import time
from skimage import measure
import os.path
from gaussenvelope import GaussEnvelope
from file_handler import point_file_reader

In [2]:
def plot_polygon(poly_lst, filename):
    if type(poly_lst) != list:
        poly_lst = [poly_lst]
    with open(filename, 'w') as csvfile:
        csvfile.write(",polygon\n")
        for ii, poly in enumerate(poly_lst):
            csvfile.write(f'{ii}, "{poly.wkt}"\n')

In [48]:
def read_from_file(filename):
    input_filename = '..\\input\\' + filename
    with open(input_filename, newline='') as csvfile:
        lines = csvfile.read().split('\n')
    points_list, points_arr = point_file_reader(lines)
    return points_list, points_arr

def mk_gaussians(filename, sigma, thresh, zoom):
    points_list, points_arr = read_from_file(filename)
    gauss_env = GaussEnvelope(points_arr)
    gauss_env.calc_gauss_grid(sigma=sigma)
    gauss_env.find_contours(thresh=thresh)
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.contour(gauss_env.xx, gauss_env.yy, gauss_env.gauss_grid, 15)
    for contour in gauss_env.contour_point_lst:
        ax.plot(contour[:, 0], contour[:, 1], 'r--')
    ax.plot(gauss_env.points[:, 0], gauss_env.points[:, 1], 'k.')
    
    x_min, y_min = points_arr.min(0)*(1-zoom)
    x_max, y_max = points_arr.max(0)*(1+zoom)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_aspect('equal')
    ax.grid()

In [50]:
from ipywidgets import interact, FloatSlider, Layout, FloatLogSlider, Dropdown
import os

interact(mk_gaussians,
         filename=Dropdown(options=os.listdir('..\\input'), description='Number:'),
         sigma=FloatLogSlider(min=-2.5, max=-1, step=0.05, value=0.01, layout=Layout(width='80%')),
         thresh=FloatLogSlider(min=-2.5, max=-0.01, step=0.1, value=0.05, layout=Layout(width='80%')),
            zoom=FloatSlider(min=0.0001, max=0.01, step=0.0004, value=0.002, layout=Layout(width='80%')));

interactive(children=(Dropdown(description='Number:', options=('bone1.csv', 'double_banana.csv', 'points_googl…